In [5]:
import urllib
import json
import pandas as pd
import time
import sys
import os

In [6]:
# Secret token
token = open("../ACCESS_TOKEN.txt", "r").read()

In [7]:
# Format a request URI for the Genius API
search_term = 'Ayla Celik'
_URL_API = "https://api.genius.com/"
_URL_SEARCH = "search?q="
querystring = _URL_API + _URL_SEARCH + urllib.parse.quote(search_term)
request = urllib.request.Request(querystring)
request.add_header("Authorization", "Bearer " + token)
request.add_header("User-Agent", "")

In [8]:
def get_songs(annotations_file, song_ids_file):
    annotation_header=['song_id','lyric','annotation']
    song_header = ['artist','song']
    try:
        annotations = pd.read_csv(annotations_file, names=annotation_header, sep='\t')
    except:
        return pd.read_csv(song_ids_file, names=song_header, sep=',')
    songs = pd.read_csv(song_ids_file, names=song_header, sep=',')
    last_id = str(annotations.song_id.values[-1])
    i = songs[songs.song.astype(str) == last_id].index[0]
    return songs[i+1:]

In [9]:
# Function to recursively search the annotation dict
# and get the text, skipping blockquotes, formatting, images, etc.

def recurse(children):
    st = ''
    for child in children:
        #print(child)
        if type(child) == dict and 'tag' in child:
            if child['tag'] == 'p' or child['tag'] == 'a':
                st += recurse(child['children'])
            elif child['tag'] == 'blockquote':
                st += '[BLOCKQUOTE]'
        elif type(child) == list:
            st += recurse(child)
        elif type(child) == str:
            st += child + ' '
    return st

In [10]:
def append_annotations(songdf, annotations_file, batchsize):
    # Get a chunk of song ids
    song_ids = songdf.head(batchsize)
    song_ids = song_ids[song_ids.song.astype(str) != 'None'].song.astype(int)

    with open(annotations_file, 'a') as f:

        for i, song_id in enumerate(song_ids):
            sys.stdout.write('\r'+str(i))

            # Format the search query using the 'referents' api
            # Given a song id, returns all of its referents (lines that have been annotated)
            # and the corresponding annotations.
            querystring = "https://api.genius.com/referents?song_id=" + str(song_id) + "&per_page=50"
            request = urllib.request.Request(querystring)
            request.add_header("Authorization", "Bearer " + token)
            request.add_header("User-Agent", "")

            # Do internet stuff
            # Send the request to Genius, and parse the response
            try:
                response = urllib.request.urlopen(request, timeout=10)
                string = response.read().decode('utf-8')
                json_obj = json.loads(string)
            except:
                print('***Failed on song id: ' + str(song_id))
                continue

            # The data we want can be found here
            # For this song there are 9 lyric/annotation pairs
            referents = json_obj['response']['referents']

            # Many songs return zero annotations. If we get annotations, parse through them
            if len(referents) > 0:
                for ref in referents:
                    lyric = ref['fragment'].replace('\n', ' ')
                    try:
                        annotation = recurse(ref['annotations'][0]['body']['dom']['children']).replace('\n', ' ')
                    except:
                        print('***Empty annotation on song id: ' + str(song_id))
                        continue
                    
                    #print(song_id)
                    sys.stdout.write('\r'+str(i)+" "+lyric)
                    f.write(str(song_id) + '\t' + lyric + '\t' + annotation + '\n')
                    
            time.sleep(0.5)

In [11]:
def main(song_ids_file, annotations_file, batch_size=1000):
    songdf = get_songs(annotations_file, song_ids_file)
    try:
        append_annotations(songdf, annotations_file, batch_size)
    except:
        os.system('say "failure"')

    os.system('say "batch done"')

In [12]:
main('song_ids_2.csv','annotations_2.tsv',10000)

248 [Tung Twista] Yo, Sinister Def  [Sinister Def] Yo, what's up money?  [Tung Twista] Yo, I want you to sing your funky flow style for your single. You know what I'm sayin'?  [Sinister Def] Indeed, I think I can do that, so what's up?  [Tung Twista] And umm... I want you to come out Tung Twisting style for me  [Sinister Def] Yeah, I see what I can do, so umm... Tung Twista  [Tung Twista] Yo  [Sinister Def] Check this outt what I've done  Drama's all in the game whether gang bang or slang  I had to do my thangown sk...***Failed on song id: 608511
257***Failed on song id: 1626260e Joe Pescie me break the trust?  Shoulda knew you were sheisty the way you lick me on my bodyou moanutiful, it's beautiful  But now I need you to do it faster, baby  Can you please? Do it faster baby, do it faster!  Damn, baby; I can't do it that fast...  But I know somebody who can, Twista!
294***Failed on song id: 1155464  Can you be my daddy? Daddy  I need a daddy, daddy  Won't you be my daddy? Daddythe hear

520***Failed on song id: 33687
521***Failed on song id: 33710
522***Failed on song id: 33663
523***Failed on song id: 33652
524***Failed on song id: 33665
525***Failed on song id: 288462
526***Failed on song id: 33735
527***Failed on song id: 1713115
528***Failed on song id: 33650
529***Failed on song id: 33745
530***Failed on song id: 33742
531***Failed on song id: 33738
532***Failed on song id: 921749
533***Failed on song id: 33711
534***Failed on song id: 33715
535***Failed on song id: 33714
536***Failed on song id: 1801541
537***Failed on song id: 1476296
538***Failed on song id: 33702
539***Failed on song id: 33741
540***Failed on song id: 909966
541***Failed on song id: 33716
542***Failed on song id: 33697
543***Failed on song id: 961995
544***Failed on song id: 33707
545***Failed on song id: 33669
546***Failed on song id: 33682
547***Failed on song id: 33722
548***Failed on song id: 1730483
549***Failed on song id: 33739
550***Failed on song id: 33677
551***Failed on song id: 33

800***Failed on song id: 16340
801***Failed on song id: 16226
802***Failed on song id: 16293
803***Failed on song id: 16358
804***Failed on song id: 16291
805***Failed on song id: 16224
806***Failed on song id: 16243
807***Failed on song id: 16225
808***Failed on song id: 747196
809***Failed on song id: 16282
810***Failed on song id: 16296
811***Failed on song id: 16246
812***Failed on song id: 16236
813***Failed on song id: 16228
814***Failed on song id: 374578
815***Failed on song id: 16364
816***Failed on song id: 16230
817***Failed on song id: 4243
818***Failed on song id: 211660
819***Failed on song id: 16370
820***Failed on song id: 2489537
821***Failed on song id: 16339
822***Failed on song id: 16341
823***Failed on song id: 1502258
824***Failed on song id: 1188435
825***Failed on song id: 16283
826***Failed on song id: 1815003
827***Failed on song id: 1815739
828***Failed on song id: 16285
829***Failed on song id: 970787
830***Failed on song id: 1553813
831***Failed on song id:

1130***Failed on song id: 944989
1131***Failed on song id: 1374618
1132***Failed on song id: 1397457
1133***Failed on song id: 1532550
1134***Failed on song id: 1842560
1135***Failed on song id: 1111378
1136***Failed on song id: 991345
1137***Failed on song id: 977175
1138***Failed on song id: 1842508
1139***Failed on song id: 1303317
1140***Failed on song id: 1194864
1141***Failed on song id: 1397972
1142***Failed on song id: 1751571
1143***Failed on song id: 2210167
1144***Failed on song id: 4011317
1145***Failed on song id: 1134050
1146***Failed on song id: 1842321
1147***Failed on song id: 1842350
1148***Failed on song id: 1842377
1149***Failed on song id: 1842609
1150***Failed on song id: 1842033
1151***Failed on song id: 1842314
1152***Failed on song id: 1842364
1153***Failed on song id: 954868
1154***Failed on song id: 1842360
1155***Failed on song id: 1072888
1156***Failed on song id: 879515
1157***Failed on song id: 2260232
1158***Failed on song id: 1655106
1159***Failed on so

1381***Failed on song id: 2052181
1382***Failed on song id: 97059
1383***Failed on song id: 1965683
1384***Failed on song id: 2080913
1385***Failed on song id: 2009947
1386***Failed on song id: 1984573
1387***Failed on song id: 1965467
1388***Failed on song id: 1942579
1389***Failed on song id: 1956286
1390***Failed on song id: 2012183
1391***Failed on song id: 1945592
1392***Failed on song id: 1943859
1393***Failed on song id: 1965309
1394***Failed on song id: 2233668
1395***Failed on song id: 1981074
1396***Failed on song id: 2085735
1397***Failed on song id: 1939995
1398***Failed on song id: 2075167
1399***Failed on song id: 3323437
1400***Failed on song id: 1998339
1401***Failed on song id: 2096531
1402***Failed on song id: 1943812
1403***Failed on song id: 2086347
1404***Failed on song id: 1957698
1405***Failed on song id: 2063481
1406***Failed on song id: 1947391
1407***Failed on song id: 1980588
1408***Failed on song id: 2049551
1409***Failed on song id: 2003376
1410***Failed on

1629***Failed on song id: 424374
1630***Failed on song id: 1973936
1631***Failed on song id: 1940728
1632***Failed on song id: 424257
1633***Failed on song id: 4130473
1634***Failed on song id: 4130631
1635***Failed on song id: 4131903
1636***Failed on song id: 4131727
1637***Failed on song id: 424340
1638***Failed on song id: 1562317
1639***Failed on song id: 1840170
1640***Failed on song id: 424339
1641***Failed on song id: 424313
1642***Failed on song id: 1839204
1643***Failed on song id: 424692
1644***Failed on song id: 424181
1645***Failed on song id: 424673
1646***Failed on song id: 936764
1647***Failed on song id: 1667629
1648***Failed on song id: 1839314
1649***Failed on song id: 1438866
1650***Failed on song id: 1590113
1651***Failed on song id: 1839374
1652***Failed on song id: 965643
1653***Failed on song id: 424256
1654***Failed on song id: 897019
1655***Failed on song id: 1218479
1656***Failed on song id: 424229
1657***Failed on song id: 4205582
1658***Failed on song id: 9

1993***Failed on song id: 424378
1994***Failed on song id: 423939
1995***Failed on song id: 424376
1996***Failed on song id: 424255
1997***Failed on song id: 4214429
1998***Failed on song id: 3015654
1999***Failed on song id: 424285
2000***Failed on song id: 1508833
2001***Failed on song id: 210342
2002***Failed on song id: 4219849
2003***Failed on song id: 424168
2004***Failed on song id: 1839282
2005***Failed on song id: 4219847
2006***Failed on song id: 423965
2007***Failed on song id: 424322
2008***Failed on song id: 1005253
2009***Failed on song id: 1459665
2010***Failed on song id: 1158841
2011***Failed on song id: 1134801
2012***Failed on song id: 864788
2013***Failed on song id: 1460694
2014***Failed on song id: 1335075
2015***Failed on song id: 1839305
2016***Failed on song id: 1350576
2017***Failed on song id: 2218427
2018***Failed on song id: 424280
2019***Failed on song id: 1520878
2020***Failed on song id: 170142
2021***Failed on song id: 1544488
2022***Failed on song id: 

2238***Failed on song id: 1227670
2239***Failed on song id: 1426716
2240***Failed on song id: 949649
2241***Failed on song id: 1479134
2242***Failed on song id: 1760044
2243***Failed on song id: 4005713
2244***Failed on song id: 1276273
2245***Failed on song id: 1013284
2246***Failed on song id: 1602420
2247***Failed on song id: 4006356
2248***Failed on song id: 1476157
2249***Failed on song id: 4005717
2250***Failed on song id: 1827010
2251***Failed on song id: 4005719
2252***Failed on song id: 1533485
2253***Failed on song id: 1796156
2254***Failed on song id: 4006043
2255***Failed on song id: 3840154
2256***Failed on song id: 1172303
2257***Failed on song id: 4006042
2258***Failed on song id: 1027726
2259***Failed on song id: 1445364
2260***Failed on song id: 3828692
2261***Failed on song id: 1016430
2262***Failed on song id: 4014287
2263***Failed on song id: 1225512
2264***Failed on song id: 1116810
2265***Failed on song id: 1462497
2266***Failed on song id: 902968
2267***Failed on

2503***Failed on song id: 1218243
2504***Failed on song id: 1464177
2505***Failed on song id: 1508896
2506***Failed on song id: 2426502
2507***Failed on song id: 2246566
2508***Failed on song id: 1019364
2509***Failed on song id: 1775405
2510***Failed on song id: 1492913
2511***Failed on song id: 1172008
2512***Failed on song id: 1161140
2513***Failed on song id: 2886877
2514***Failed on song id: 1452088
2515***Failed on song id: 1205538
2516***Failed on song id: 1381314
2517***Failed on song id: 1691457
2518***Failed on song id: 1475635
2519***Failed on song id: 1977260
2520***Failed on song id: 2903962
2521***Failed on song id: 1190426
2522***Failed on song id: 1372089
2523***Failed on song id: 2886891
2524***Failed on song id: 1229001
2525***Failed on song id: 480847
2526***Failed on song id: 1991005
2527***Failed on song id: 892060
2528***Failed on song id: 1014859
2529***Failed on song id: 1032575
2530***Failed on song id: 1278807
2531***Failed on song id: 1585597
2532***Failed on

In [1]:
!wc -l annotations_2.tsv

  183841 annotations_2.tsv
